In [1]:
import pandas as pd
import vk_api
import requests

In [2]:
def get_id_vk(name):
    id_vk=vk_session.method('utils.resolveScreenName', {'screen_name': name})
    return id_vk['object_id']

In [3]:
def force(f, delay=0.5):
  """При неудачном запросе сделать паузу и попробовать снова"""
  def tmp(*args, **kwargs):
    while True:
      try:
        res = f(*args, **kwargs)
        break
      except KeyError:
        time.sleep(delay)
    return res
  return tmp

In [4]:
vk_session=vk_api.VkApi('tel', 'parol', app_id=2685278)
vk_session.auth()
vk=vk_session.get_api()

In [5]:
tables = pd.read_csv("data_name.csv")
vk_id=[]
for i in tables.iloc:
    vk_id.append(get_id_vk(i['id']))
tables['vk_id']=vk_id
tables['level']=0

In [6]:
tables

,ФИО,id,vk_id,level
0,Алексеева Екатерина,kate_al02,181400458,0
1,Бирюков Виктор,iktovr,136004593,0
2,Васильев Даниил,da_mil_ka,151143124,0
3,Воронов Кирилл,id248826936,248826936,0
4,Гребенков Дмитрий,irnevoir,172224066,0
5,Ляшун Дмитрий,id356135268,356135268,0
6,Новиков Никита,rapcor,118466021,0
7,Пивницкий Даниэль,slast1_one,625131977,0
8,Уваров Андрей,anonumovich,206240342,0
9,Шубин Григорий,mr.kekess,196085241,0


In [7]:
tables.drop(columns = ['id'],axis = 1, inplace=True) 
tables

,ФИО,vk_id,level
0,Алексеева Екатерина,181400458,0
1,Бирюков Виктор,136004593,0
2,Васильев Даниил,151143124,0
3,Воронов Кирилл,248826936,0
4,Гребенков Дмитрий,172224066,0
5,Ляшун Дмитрий,356135268,0
6,Новиков Никита,118466021,0
7,Пивницкий Даниэль,625131977,0
8,Уваров Андрей,206240342,0
9,Шубин Григорий,196085241,0


In [16]:
@force
def get_friend(vk_id):
    url = f"https://api.vk.com/method/friends.get?user_id={vk_id}&fields=bdate&access_token=vk1.a.2-GlqlG1jHMD2QCIrmvKkBOC0GL_gmIX2RQVkHfQ4gbN0SM0hctW7IskhD4xckp1zYxvjoxGNmQrjR3b0otsQSzfX_YAz915t13V8bKkZb2l1yNKZuMkYEKdHshC0xJrFtNvshudkBJUBv5HMqxA8BQ3C9ffif671IB4JkbSU-MRJaYzE1u0t354mzsj9oFVqBziUMYdXoB2B0KJhHu6Fg&v=5.154"
    data = requests.get(url).json()
    return data

In [18]:
level_1={'ФИО':[], 'vk_id':[]}
dict_for_graph={}
for i in tables.iloc:
    friends=[]
    data=get_friend(i['vk_id'])
    if 'error' not in data:
        for j in data['response']['items']:
            name=j['last_name']+' '+j['first_name']
            vk_id=j['id']
            friends.append(vk_id)
            level_1['ФИО'].append(name)
            level_1['vk_id'].append(vk_id)
    dict_for_graph[i['vk_id']]=friends
        

In [19]:
df_level_1 = pd.DataFrame(level_1)
df_level_1['level']=1
df_level_1.drop_duplicates(inplace=True)

In [20]:
df_level_1

,ФИО,vk_id,level
0,Зилов Виктор,4376118,1
1,Гончаров Константин,4572608,1
2,Мугинов Руслан,9751803,1
3,Ii Владислав,20088675,1
4,Алексеев Владислав,82156976,1
...,...,...,...
924,Крестьянников Дмитрий,591996670,1
925,Литковский Никита,596120747,1
926,Pivnitskiy Daniel,625131977,1
927,Korolev Ilya,672453778,1


In [21]:
level_2={'ФИО':[], 'vk_id':[]}
for i in df_level_1.iloc:
    friends=[]
    data=get_friend(i['vk_id'])
    if 'error' not in data:
        for j in data['response']['items']:
            name=j['last_name']+' '+j['first_name']
            vk_id=j['id']
            friends.append(vk_id)
            level_2['ФИО'].append(name)
            level_2['vk_id'].append(vk_id)
    dict_for_graph[i['vk_id']]=friends

In [22]:
df_level_2 = pd.DataFrame(level_2)
df_level_2['level']=2
df_level_2.drop_duplicates(inplace=True)
df_level_2

,ФИО,vk_id,level
0,DELETED,14514,2
1,Иванов Владимир,133501,2
2,Гайсина Наталия,237181,2
3,Zakharova Nadya,508316,2
4,Колодина Анастасия,508941,2
...,...,...,...
116137,Нет Нет,642816502,2
116138,Mifi Don,646582081,2
116140,Htet Min,703428141,2
116143,Рей Армяне,754843363,2


In [47]:
df_people=pd.concat([tables, df_level_1, df_level_2])
df_people.drop_duplicates(inplace=True)

In [48]:
ran=[i for i in range(0, df_people.shape[0])]
df_people['id']=ran
df_people=df_people[['id', 'vk_id', 'ФИО', 'level']]
df_people.to_csv('people.csv')

In [49]:
df_people

,id,vk_id,ФИО,level
0,0,181400458,Алексеева Екатерина,0
1,1,136004593,Бирюков Виктор,0
2,2,151143124,Васильев Даниил,0
3,3,248826936,Воронов Кирилл,0
4,4,172224066,Гребенков Дмитрий,0
...,...,...,...,...
116137,97610,642816502,Нет Нет,2
116138,97611,646582081,Mifi Don,2
116140,97612,703428141,Htet Min,2
116143,97613,754843363,Рей Армяне,2


In [50]:
start_pos=[]
target_pos=[]
for k,v in dict_for_graph.items():
    temp=[k]*len(v)
    start_pos.extend(temp)
    target_pos.extend(v)

In [54]:
dict_for_graph_2={'start_pos':start_pos, 'target_pos':target_pos}

In [55]:
df_dict_for_graph=pd.DataFrame(dict_for_graph_2)

In [56]:
df_dict_for_graph=pd.merge(df_dict_for_graph, df_people, left_on='start_pos', right_on='vk_id')
df_dict_for_graph=df_dict_for_graph.drop(columns=['start_pos', 'vk_id', 'ФИО'])
df_dict_for_graph=df_dict_for_graph.rename(columns = {'id':' start_pos'})
df_dict_for_graph=pd.merge(df_dict_for_graph, df_people, left_on='target_pos', right_on='vk_id')
df_dict_for_graph=df_dict_for_graph.drop(columns=['target_pos', 'vk_id', 'ФИО', 'level_x', 'level_y'])
df_dict_for_graph=df_dict_for_graph.rename(columns = {'id':' target_pos'})
df_dict_for_graph

,start_pos,target_pos
0,1,152
1,1,6085
2,418,152
3,418,6085
4,27606,152
...,...,...
231612,24029,97612
231613,831,97613
231614,24029,97613
231615,831,97614


In [57]:
df_dict_for_graph.to_csv('data_for_graph.csv')